In [1]:
import sys

import time
import numpy as np
import sys,os
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from hfnet.settings import EXPER_PATH
from notebooks.utils import plot_images, plot_matches, add_frame

import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import logging
from tqdm import tqdm
import numpy as np
from scipy.spatial.transform import Rotation as R, Slerp 
from numpy.linalg import inv, norm
logging.basicConfig(format='[%(asctime)s %(levelname)s] %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
from hfnet.datasets import get_dataset  # noqa: E402
from hfnet.evaluation.loaders import export_loader  # noqa: E402
from hfnet.settings import EXPER_PATH  # noqa: E402
from scipy import spatial


tf.contrib.resampler  # import C++ op
class HFNet:
    def __init__(self, model_path, outputs):
        self.session = tf.Session()
        self.image_ph = tf.placeholder(tf.float32, shape=(None, None, 3))

        net_input = tf.image.rgb_to_grayscale(self.image_ph[None])
        tf.saved_model.loader.load(
            self.session, [tag_constants.SERVING], str(model_path),
            clear_devices=True,
            input_map={'image:0': net_input})

        graph = tf.get_default_graph()
        self.outputs = {n: graph.get_tensor_by_name(n+':0')[0] for n in outputs}
        self.nms_radius_op = graph.get_tensor_by_name('pred/simple_nms/radius:0')
        self.num_keypoints_op = graph.get_tensor_by_name('pred/top_k_keypoints/k:0')
    def inference(self, image, nms_radius=4, num_keypoints=1000):
        inputs = {
            self.image_ph: image[..., ::-1].astype(np.float),
            self.nms_radius_op: nms_radius,
            self.num_keypoints_op: num_keypoints,
        }
        return self.session.run(self.outputs, feed_dict=inputs)

model_path = Path(EXPER_PATH, 'saved_models/hfnet')
outputs = ['global_descriptor', 'keypoints', 'local_descriptors','scores']
hfnet = HFNet(model_path, outputs)

/home/pang/software/hfnet_venv/venv_cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pang/software/hfnet_venv/venv_cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pang/software/hfnet_venv/venv_cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pang/software/hfnet_venv/

In [4]:
%load_ext autoreload
%autoreload 2
db_path =  "/media/pang/Elements1/dataset/south-building/images/"
query_path ="/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/" 

db_list = os.listdir(db_path)
query_list = os.listdir(query_path)
query_list.sort()
db_list.sort()

db_image = lambda n: cv2.imread(db_path + n)[:, :, ::-1]
query_image = lambda n: cv2.imread(query_path + n)[:, :, ::-1]
images_db = [db_image(frame) for frame in db_list]
images_query = [query_image(frame) for frame in query_list]

# db = [hfnet.inference(i) for i in images_db]
# global_index = np.stack([d['global_descriptor'] for d in db])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
## export feature for SfM
export_path = '/media/pang/Elements1/dataset/south-building/hf_feature/'
if not os.path.isdir(export_path):
    os.mkdir(export_path)


for i in range(len(images_db)):
    data = hfnet.inference(images_db[i])
    export = {
        'keypoints': data['keypoints'],
        'local_descriptors': data['local_descriptors'],
        'global_descriptor':data['global_descriptor'],
        'scores':data['scores']
    }
    name = export_path + db_list[i][:-4]
    np.savez(f'{name}.npz', **export)